In [ ]:
%load_ext autoreload
%autoreload 2

import open3d as o3d
import numpy as np
import copy
from os import listdir
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from winsound import Beep
import time
import glob

import alexutils as alx



In [ ]:


src = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\different_slams\mulls_results\5_binary\result\pose_b_lo_xxx_id.txt"
arr = get_transform_matrices_from_kitti(src)[1:]

mat = np.loadtxt(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Blender\110_gt.txt")
mat[0,3] = mat[0,3] - 0.5

new_arr = [np.matmul(a, mat).flatten()[0:12] for a in arr]


In [ ]:
combined_transformations = [' '.join([str(round(x, 5)) for x in t.tolist()])+ "\n" for t in new_arr]

with open(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\ground_truth_from_mulls_kitti.txt", 'w') as fp:
    fp.write(''.join(combined_transformations))

Compare MULLS with KISS ICP

In [ ]:

src = r"C:\Users\amoff\results\2023-09-26_12-32-48\pcd_poses_kitti.txt"
pcd_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\more_frames\pcd\\"
pcd_target_merged_file = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\more_frames\merged\merged_vehicle_more_frames_kiss_icp3.pcd"

transform_matrices = alx.get_transform_matrices_from_kitti(src)
print(f"Number of transform matrices {len(transform_matrices)}")

filenames = [x.split("\\")[-1] for x in glob.glob(pcd_folder + "*.pcd")]
print(f"Number of pcd files: {len(filenames)}")
pcl = o3d.geometry.PointCloud() #Empty point cloud

for idx, file in enumerate(filenames):
    point_cloud = o3d.io.read_point_cloud(pcd_folder + file)
    point_cloud = alx.remove_ground(point_cloud, 0.5)
    point_cloud.transform(transform_matrices[idx])
    pcl += point_cloud
    print(str(idx))

# print(f"Number of points with ground {len(pcl.points)}")
no_ground_pcd = pcl
print(f"Number of points without ground {len(no_ground_pcd.points)}")
no_ground_pcd = no_ground_pcd.voxel_down_sample(voxel_size = 0.1)
print(f"Number of points with downsampling {len(no_ground_pcd.points)}")
alx.draw_point_clouds([no_ground_pcd])
o3d.io.write_point_cloud(pcd_target_merged_file, no_ground_pcd, write_ascii = False)

In [ ]:
def merge_pcds(transform_matrices, pcd_folder, pcd_target_merged_file):
    filenames = glob.glob(pcd_folder + "*.pcd")
    print(filenames)
    pcl = o3d.geometry.PointCloud() #Empty point cloud

    for idx, file in enumerate(filenames):
        point_cloud = o3d.io.read_point_cloud(file)
        point_cloud = alx.remove_ground(point_cloud, 0.5)
        point_cloud.transform(transform_matrices[idx])
        pcl += point_cloud
        print(str(idx))

    
    no_ground_pcd = pcl
    print(f"Number of points without ground {len(no_ground_pcd.points)}")
    no_ground_pcd = no_ground_pcd.voxel_down_sample(voxel_size = 0.5)
    print(f"Number of points with downsampling {len(no_ground_pcd.points)}")
    o3d.io.write_point_cloud(pcd_target_merged_file, no_ground_pcd, write_ascii = False)

In [ ]:
def apply_transformation_merge_and_save(transform_matrices, second_transformation, pcd_folder, target_folder):
    filenames = [x.split("\\")[-1] for x in glob.glob(pcd_folder + "*.pcd")]
    pcl = o3d.geometry.PointCloud()

    for idx, file in enumerate(filenames):
        point_cloud = o3d.io.read_point_cloud(pcd_folder + file)
        # point_cloud = alx.remove_ground(point_cloud, 0.5)
        point_cloud.transform(transform_matrices[idx])
        pcl += point_cloud
        print(str(idx))

    pcl.transform(second_transformation)
    no_ground_pcd = pcl.voxel_down_sample(voxel_size = 0.1)
    print("Write merged point cloud to " + target_folder + file)
    o3d.io.write_point_cloud(target_folder + file, no_ground_pcd, write_ascii = False)




In [ ]:
def apply_transformation_and_save(transform_matrices, second_transformation, pcd_folder, target_folder):
    filenames = [x.split("\\")[-1] for x in glob.glob(pcd_folder + "*.pcd")]

    for idx, file in enumerate(filenames):
        point_cloud = o3d.io.read_point_cloud(pcd_folder + file)
        point_cloud = alx.remove_ground(point_cloud, 0.5)
        point_cloud.transform(transform_matrices[idx])
        point_cloud.transform(second_transformation)
        # no_ground_pcd = point_cloud.voxel_down_sample(voxel_size = 0.1)
        no_ground_pcd = point_cloud
        o3d.io.write_point_cloud(target_folder + file, no_ground_pcd, write_ascii = False)
        print(str(idx))

    

In [ ]:
src = r"C:\Users\amoff\results\2023-09-26_12-32-48\pcd_poses_kitti.txt"
pcd_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\more_frames\pcd\\"
pcd_target_merged_file = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\more_frames\merged\merged_vehicle_more_frames_kiss_icp3.pcd"

src = r"C:\Users\amoff\results\2023-09-22_02-26-55\pcd_without_nan_renamed_binary_poses_kitti.txt"
pcd_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\pcd_without_nan\\"
pcd_target_merged_file = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\merged_files\merged_vehicle_frames_kiss_icp_with_ground.pcd"

transform_matrices = alx.get_transform_matrices_from_kitti(src)
merge_pcds(transform_matrices, pcd_folder, pcd_target_merged_file)

In [ ]:
src = r"C:\Users\amoff\results\2023-09-22_02-26-55\pcd_without_nan_renamed_binary_poses_kitti.txt"
second_transformation = np.loadtxt(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\merged_files\merged_vehicle_frames_kiss_icp_transformation.txt")

pcd_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\pcd_without_nan\\"
pcd_target_merged_file = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\pcd_registered_to_dirty2_no_downsample\\"


transform_matrices = alx.get_transform_matrices_from_kitti(src)
apply_transformation_and_save(transform_matrices, second_transformation, pcd_folder, pcd_target_merged_file)

In [ ]:
src = r"C:\Users\amoff\results\2023-09-22_02-26-55\pcd_without_nan_renamed_binary_poses_kitti.txt"
pcd_folder = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\pcd_without_nan\\"
pcd_target_merged_file = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\registered_pcds\\"

second_transformation = np.loadtxt(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\merged_files\merged_vehicle_frames_kiss_icp_transformation.txt")

transform_matrices = alx.get_transform_matrices_from_kitti(src)
apply_transformation_merge_and_save(transform_matrices, second_transformation, pcd_folder, pcd_target_merged_file)

Normalize the drone data

In [ ]:
source = o3d.io.read_point_cloud(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\registered_pcds\merged_vehicle_frames_kiss_icp_no_ground_registered.pcd")
target = o3d.io.read_point_cloud(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\registered_pcds\intersection_drone_no_ground_registered.pcd")



source , target = alx.pc_norm(source, target)
alx.draw_point_clouds([source, target])

o3d.io.write_point_cloud(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\registered_pcds\merged_vehicle_frames_kiss_icp_no_ground_registered_normalized.pcd", source)
o3d.io.write_point_cloud(r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\registered_pcds\intersection_drone_no_ground_registered_normalized.pcd", target)
